In [1]:
## Carga Data INGRESO X MATERIA

In [2]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [5]:
path_raw = "../data/raw" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

In [6]:
#creacion dataframe con los datos de INGRESOS por MATERIA Penal de los años 2015 a 2019

dataframes = []

for archivo in archivos:
    if archivo.find("Ingresos por Materia Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', dtype = 'unicode',low_memory = True)
        dataframes.append(df) 
        
df_ingresos_materia = pd.concat(dataframes, axis = 0)


In [7]:
df_ingresos_materia.columns

Index(['N°', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'COD. MATERIA', 'MATERIA', 'FECHA INGRESO', 'MES INGRESO',
       'AÑO INGRESO', 'TOTAL INGRESOS POR MATERIAS',
       '(*)Se agregó columna total de ingresos, dado que en algunas causas, la materia se repite (error de tramitación)',
       'TOTAL INGRESOS POR MATERIAS(*)'],
      dtype='object')

In [8]:
df_ingresos_materia.drop(['N°','MES INGRESO','AÑO INGRESO','TOTAL INGRESOS POR MATERIAS(*)','TOTAL INGRESOS POR MATERIAS'],
                         axis = 'columns', inplace = True)

df_ingresos_materia.drop(['(*)Se agregó columna total de ingresos, dado que en algunas causas, la materia se repite (error de tramitación)'], 
                         axis='columns',inplace=True)

#df_ingresos_materia = df_ingresos_materia.dropna()

In [9]:
# TRANSFORMAMOS DE FLOAT A INTEGER

df_ingresos_materia['COD. CORTE'] = df_ingresos_materia['COD. CORTE'].fillna(0).astype(np.int16)
df_ingresos_materia['COD. TRIBUNAL'] = df_ingresos_materia['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_ingresos_materia['COD. MATERIA'] = df_ingresos_materia['COD. MATERIA'].fillna(0).astype(np.int16)

In [11]:
# Transformamos fechas

df_ingresos_materia['FECHA INGRESO'] = df_ingresos_materia['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 3213259/3213259 [00:12<00:00, 254693.07it/s]


In [12]:
# Elimino espacios en las columnas tipo objetos

df_ingresos_materia = df_ingresos_materia.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 9/9 [00:10<00:00,  1.20s/it]


In [13]:
# Elimino tildes 

cols = df_ingresos_materia.select_dtypes(include = ["object"]).columns
df_ingresos_materia[cols] = df_ingresos_materia[cols].progress_apply(cleandata.elimina_tilde) 

100%|██████████| 5/5 [00:40<00:00,  8.15s/it]


In [14]:
# Categorizacion

df_ingresos_materia['CORTE'] = df_ingresos_materia['CORTE'].astype('category')
df_ingresos_materia['TIPO CAUSA'] = df_ingresos_materia['TIPO CAUSA'].astype('category')

In [15]:
df_ingresos_materia.reset_index(inplace = True)
df_ingresos_materia

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,NaT
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",NaT
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,NaT
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,NaT
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,NaT
...,...,...,...,...,...,...,...,...,...,...
3213254,631764,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,99-2015,Ordinaria,12082,AMENAZASA CARABINEROS (ART. 417 COD. J.MILITAR).,NaT
3213255,631765,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,100-2015,Ordinaria,810,ROBO EN LUGAR NO HABITADO.,NaT
3213256,631766,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,101-2015,Ordinaria,808,ROBO EN BIENES NACIONALES DE USO PUBLICO O SIT...,NaT
3213257,631767,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,102-2015,Ordinaria,621,VIOLACION DE MENOR DE 14 AÑOS.,NaT


In [17]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_ingresos_materia.to_feather(f'{path_interim}/IngresosMateria_feather')